# Lab Assignment Six: Convolutional Network Architectures



## Dataset Selection

Ships in Satellite Imagery
https://www.kaggle.com/datasets/rhammell/ships-in-satellite-imagery

Group: Nick Benso & Benjamin Kuo

In [2]:
from matplotlib import pyplot as plt
from skimage import color
from skimage import io
import numpy as np
import pandas as pd
import os

#image name format: {label} _ {scene id} _ {longitude} _ {latitude}.png
#label: 0-no ship(includes incomplete ships), 1-ship

root = r"data\\shipsnet\\shipsnet\\"
testfile = r"0__20150718_184300_090b__-122.35324421973536_37.772113980272394.png"

def loadImg(dir, imgFile):
    img = io.imread(dir + imgFile)
    return color.rgb2gray(img)

def filenameParse(filename):
    repl_delim = '__'
    str = filename.replace(".png", repl_delim)
    return str.split('__')

In [3]:

#read in all images into np arrays
img = []
label = []
sceneID = []
long = []
lat = []

directory = os.fsencode(root)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    count = 0
    if(filename.endswith(".png")):
        img.append(loadImg(root, filename).flatten())
        parsed = filenameParse(filename)
        label.append(parsed[0])
        sceneID.append(parsed[1])
        long.append(parsed[2].split('_')[0])
        lat.append(parsed[2].split('_')[1])
        continue
    else:
        continue
        
print(type(img[0]))


<class 'numpy.ndarray'>


In [5]:
#load np arrays into a dataframe
d = {'data': img, 'label': label, 'sceneID': sceneID, 'longitude': long, 'latitude': lat}
df = pd.DataFrame(data=d)

df.label = df.label.astype(int).astype(bool)
df.longitude = df.longitude.astype(np.float64)
df.latitude = df.latitude.astype(np.float64)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   data       4000 non-null   object 
 1   label      4000 non-null   bool   
 2   sceneID    4000 non-null   object 
 3   longitude  4000 non-null   float64
 4   latitude   4000 non-null   float64
dtypes: bool(1), float64(2), object(2)
memory usage: 129.0+ KB


,data,label,sceneID,longitude,latitude
0,"[0.44731176470588235, 0.4428396078431372, 0.42...",False,20150718_184300_090b,-122.353244,37.772114
1,"[0.25246235294117647, 0.2510635294117647, 0.25...",False,20150718_184300_090b,-122.384586,37.763521
2,"[0.3374623529411765, 0.42707019607843133, 0.51...",False,20150718_184300_090b,-122.404775,37.807104
3,"[0.41511372549019604, 0.4299666666666666, 0.41...",False,20150718_184300_090b,-122.426639,37.809132
4,"[0.6346462745098039, 0.6262525490196079, 0.590...",False,20150720_184302_0906,-122.218938,37.871723


## Preparation (3 points total)  
[1.5 points] Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.

[1.5 points] Choose the method you will use for dividi
ng your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your cross validation method is a realistic mirroring of how an algorithm would be used in practice. 

## Modeling (6 points total)
[1.5 points]  Setup the training to use data expansion in Keras (also called data augmentation). Explain why the chosen data expansion techniques are appropriate for your dataset. You can use the keras ImageGenerator as a pre-processing step OR in the optimization loop. You can also use the Keras-cv augmenter (a separate package: https://keras.io/keras_cv/Links to an external site.)

[2 points] Create a convolutional neural network to use on your data using Keras. Investigate at least two different convolutional network architectures (and investigate changing some parameters of each architecture such as the number of filters--at minimum have two variations of each network for a total of four models trained). Use the method of train/test splitting and evaluation metric that you argued for at the beginning of the lab. Visualize the performance of the training and validation sets per iteration (use the "history" parameter of Keras). Be sure that models converge.

[1.5 points] Visualize the final results of the CNNs and interpret/compare the performances. Use proper statistics as appropriate, especially for comparing models. 

[1 points] Compare the performance of your convolutional network to a standard multi-layer perceptron (MLP) using the receiver operating characteristic and area under the curve. Use proper statistical comparison techniques.  


## Exceptional Work (1 points total)
You have free reign to provide additional analyses. 

One idea (required for 7000 level students): Use transfer learning to pre-train the weights of your initial layers of your CNN. Compare the performance when using transfer learning to training without transfer learning (i.e., compare to your best model from above) in terms of classification performance. 